In [ ]:
import os
# 【硬性要求】加载环境变量
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_community.document_loaders import PyPDFLoader
# 导入递归字符文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ==========================================
# 1. 准备数据源 (承接上一节课的流程)
# ==========================================
# 假设我们有一个名为 handbook.pdf 的文件
PDF_PATH = "data/handbook.pdf"

# 占位逻辑：如果本地没有文件，生成一个测试用例以保证代码可运行
if not os.path.exists(PDF_PATH):
    os.makedirs("data", exist_ok=True)
    with open(PDF_PATH, "w") as f:
        # 我们不用 FPDF 生成真实 PDF 也可以，这里仅为示意，
        # 在真实环境中请确保 handbook.pdf 存在。
        pass 

try:
    loader = PyPDFLoader(PDF_PATH)
    # 按页物理分割，返回 Document 列表
    pages = loader.load_and_split() 
    print(f"✅ 成功加载 PDF，共计 {len(pages)} 页。")

    # ==========================================
    # 2. 实施分块策略 (Chunking Strategy)
    # ==========================================
    # 实例化文本分割器
    # chunk_size=200: 每块尽量包含 200 个字符
    # chunk_overlap=50: 相邻块之间保留 50 个字符的重叠冗余
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200, 
        chunk_overlap=50
    )
    
    # 传入页面列表，执行深入分割
    chunks = text_splitter.split_documents(pages)
    
    # 验证分块结果，例如 3 页的 PDF 被切分成了数十个小块
    print(f"🔪 分块完成！原始页面被切分成了 {len(chunks)} 个独立的 Chunk。")

    # ==========================================
    # 3. 验证重叠与元数据保留 (科研重点观察)
    # ==========================================
    if len(chunks) >= 2:
        print("\n=== 观察 Chunk 0 ===")
        print(f"内容: {chunks[0].page_content}")
        # 观察元数据是否完好传递（用于后续溯源引用）
        print(f"元数据: {chunks[0].metadata}") 
        
        print("\n=== 观察 Chunk 1 ===")
        print(f"内容: {chunks[1].page_content}")
        print(f"元数据: {chunks[1].metadata}")
        
        print("\n💡 【私教提示】对比 Chunk 0 的结尾和 Chunk 1 的开头。")
        print("你会发现有一段重复的话 (Overlap)，这正是为了保证检索时语义不被生硬切断！")

except Exception as e:
    print(f"执行异常，请确保 '{PDF_PATH}' 是一个有效的 PDF 文件。错误信息: {e}")